<a href="https://colab.research.google.com/github/JohnYechanJo/Novo-Nordisk_Synthetic-Image-Usage/blob/main/2_nd_demo(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Train on both domain(Kermany and OCT5K) and test the gap

#1.1 Training Simple CNN classifier


In [1]:
!pip install kagglehub torch torchvision transformers diffusers accelerate datasets xformers pytorch-fid pandas
!pip install torchmetrics --upgrade torch-fidelity
import os
import gc
import torch
import numpy as np
import random
import kagglehub
import torch.nn as nn
import time
from PIL import Image
import pandas as pd
from torch.utils.data import Sampler
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import ViTModel, CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline
from accelerate import Accelerator
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from pytorch_fid import fid_score
from sklearn.metrics import accuracy_score, classification_report
# Set random seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# GPU setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Memory cleanup function
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

#1.2 Dataset remerge(SKIP)
Place the code file in the extracted AMD/DME/Normal directory, modify the `prefix` accordingly, and then run it. url: `https://drive.google.com/uc?id=1ZK_qMNuy53Nfxjo2vShHMFz2LyGwYAuW`
then upload the file on drive

In [ ]:
import shutil, os
# set this
prefix = 'Normal'
def merge_all_image_dirs_under(root_dir, dst_dir, prefix=None):
    os.makedirs(dst_dir, exist_ok=True)
    global_index = 0

    src_dirs = [
        os.path.join(root_dir, d)
        for d in os.listdir(root_dir)
        if os.path.isdir(os.path.join(root_dir, d))
    ]

    for index, src_dir in enumerate(src_dirs):
        for fname in os.listdir(src_dir):
            if fname.lower().endswith(('.tiff')):
                src_path = os.path.join(src_dir, fname)

                if prefix:
                    new_fname = f"{prefix}_{global_index}.TIFF"
                else:
                    new_fname = f"{global_index}.TIFF"

                dst_path = os.path.join(dst_dir, new_fname)
                shutil.copy(src_path, dst_path)
                global_index += 1
                #print(f"{global_index} - copied from {src_dir}")

    print(f"Successfully merged: {global_index} images in total!")



root_dir = os.getcwd() + f"/{prefix}"
dst_dir = os.getcwd() + f"/merged_{prefix}"

merge_all_image_dirs_under(root_dir, dst_dir, prefix)


#1.3 Dataset load and Path set

In [2]:
from google.colab import drive
drive.mount('/content/drive')# mount drive

# download dataset
path = kagglehub.dataset_download("paultimothymooney/kermany2018")
loadpath = os.path.join(path, "OCT2017 /train")
train_path_dme = os.path.join(loadpath, "DME")
train_path_normal = os.path.join(loadpath, "NORMAL")
lp = os.path.join(path, "OCT2017 /val")
val_path_dme = os.path.join(lp, "DME")
val_path_normal = os.path.join(lp, "NORMAL")
lpp = os.path.join(path, "OCT2017 /test")
test_path_dme = os.path.join(lpp, "DME")
test_path_normal = os.path.join(lpp, "NORMAL")
# get OCT5K from drive /content/drive/MyDrive
target_dme = '/content/drive/MyDrive/OCT5K/DME'
target_normal = '/content/drive/MyDrive/OCT5K/Normal'
# 1600:1600->train 160(80:80)*20 / 8:8->val / 242:242 origin test
# target test 1000:1000
# define
'''
bsz = 160
half_bsz = bsz // 2
batch_num = 20
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'\nbsz = 160\nhalf_bsz = bsz // 2\nbatch_num = 20\n'

#1.4 Dataloader Definition

In [3]:
def img_transform():
    return transforms.Compose([
        transforms.Lambda(lambda img: img.crop((0, 100, 768, 400)) if img.size[0] >= 768 and img.size[1] >= 400 else img),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    #3，224，224
class OCTBinaryDataset(Dataset):
    def __init__(self, dme_dir, normal_dir, transform=None, max_per_class=None):
        self.transform = transform
        self.samples = []

        dme_images = sorted([os.path.join(dme_dir, f) for f in os.listdir(dme_dir)
                             if f.lower().endswith(('.jpg', '.jpeg', '.png','.tiff'))])
        normal_images = sorted([os.path.join(normal_dir, f) for f in os.listdir(normal_dir)
                                if f.lower().endswith(('.jpg', '.jpeg', '.png','.tiff'))])

        if max_per_class:
            dme_images = dme_images[:max_per_class]
            normal_images = normal_images[:max_per_class]

        self.dme_paths = dme_images
        self.normal_paths = normal_images
        assert len(self.dme_paths) > 0, f"No images found in {dme_dir}"
        assert len(self.normal_paths) > 0, f"No images found in {normal_dir}"
        self.samples += [(path, 1) for path in dme_images]      # DME: label 1
        self.samples += [(path, 0) for path in normal_images]   # NORMAL: label 0

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label
class BalancedBinaryBatchSampler(Sampler):
    def __init__(self, dme_len, normal_len, batch_size, num_batches=None, seed = 123):
        assert batch_size % 2 == 0, "Batch size must be divisible by 2"
        self.dme_len = dme_len
        self.normal_len = normal_len
        self.half_batch = batch_size // 2
        self.num_batches = num_batches or min(dme_len, normal_len) // self.half_batch
        self.seed = seed
        assert self.dme_len >= self.half_batch * self.num_batches, "Not enough DME samples"
        assert self.normal_len >= self.half_batch * self.num_batches, "Not enough NORMAL samples"


    def __iter__(self):
        rng = random.Random(self.seed)
        dme_indices = list(range(self.dme_len))
        normal_indices = list(range(self.normal_len))
        rng.shuffle(dme_indices)
        rng.shuffle(normal_indices)

        for i in range(self.num_batches):
            dme_batch = dme_indices[i*self.half_batch:(i+1)*self.half_batch]
            normal_batch = normal_indices[i*self.half_batch:(i+1)*self.half_batch]
            yield dme_batch + [j + self.dme_len for j in normal_batch]  # shift normal index

    def __len__(self):
        return self.num_batches

def build_train_dataloader(dme_dir, normal_dir, batch_size=160, num_batches=20, transform=None, max_per_class=None):
    transform = transform or img_transform()
    dataset = OCTBinaryDataset(dme_dir, normal_dir, transform=transform, max_per_class=max_per_class)
    dme_len = len(dataset.dme_paths)
    normal_len = len(dataset.normal_paths)
    sampler = BalancedBinaryBatchSampler(dme_len, normal_len, batch_size=batch_size, num_batches = num_batches)
    def balanced_collate_fn(batch):
        random.shuffle(batch)
        imgs, labels = zip(*batch)
        return torch.stack(imgs), torch.tensor(labels)
    loader = DataLoader(dataset, batch_sampler=sampler,
                        collate_fn=balanced_collate_fn,
                        num_workers=2, pin_memory=True)
    return loader
def build_valandtest_dataloader(dme_dir, normal_dir, batch_size, transform=None, max_per_class=None):
    transform = transform or img_transform()
    dataset = OCTBinaryDataset(dme_dir, normal_dir, transform=transform, max_per_class=max_per_class)
    dme_len = len(dataset.dme_paths)
    normal_len = len(dataset.normal_paths)
    loader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=False,
                        num_workers=2, pin_memory=True)
    return loader

train_loader = build_train_dataloader(
    dme_dir= train_path_dme ,
    normal_dir= train_path_normal,
    batch_size=160,
    num_batches = 20,
    transform=img_transform(),
    max_per_class=1600
)

val_loader = build_valandtest_dataloader(
    dme_dir= val_path_dme ,
    normal_dir= val_path_normal,
    batch_size=8,
    transform=img_transform(),
    max_per_class = 8
)

test_loader_origin = build_valandtest_dataloader(
    dme_dir= test_path_dme ,
    normal_dir= test_path_normal,
    batch_size=242,
    transform=img_transform(),
    max_per_class = 242
)

# may cost extra time to load when test for the first time
test_loader_target = build_valandtest_dataloader(
    dme_dir= target_dme ,
    normal_dir= target_normal,
    batch_size=100,
    transform=img_transform(),
    max_per_class = 1000
)


#1.5 Clssifier Definition

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # (3,224,224) -> (16,224,224)
            nn.ReLU(),
            nn.MaxPool2d(2),                            # -> (16,112,112)

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # -> (32,112,112)
            nn.ReLU(),
            nn.MaxPool2d(2),                             # -> (32,56,56)

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # -> (64,56,56)
            nn.ReLU(),
            nn.MaxPool2d(2)                              # -> (64,28,28)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),             # -> (64×28×28)
            nn.Linear(64*28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 2)         # 2-class classification
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x
class Classifier(nn.Module):
    def __init__(self, train_loader, val_loader):
        super().__init__()
        self.model = SimpleCNN().cuda()
        self.best_acc = 0
        self.model_path = "/content/drive/MyDrive/Classifier_model/best_model.pth"
        self.train_loader = train_loader
        self.val_loader = val_loader


    def load_model(self):
        if os.path.exists(self.model_path):
            print(f"Found saved model at {self.model_path}, loading...")
            checkpoint = torch.load(self.model_path,map_location='cuda' if torch.cuda.is_available() else 'cpu')
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.model.eval()
            return True
        return False

    def train_val(self):
      if self.load_model():
        print("Skipping training, using loaded model.")
      else:
        print("No saved model found, starting training.")
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        loss = nn.CrossEntropyLoss()
        epochs = 10
        for epoch in range(epochs):
            print(f"\nEpoch {epoch + 1}/{epochs}")
            # train
            self.model.train()
            for i, data in enumerate(self.train_loader):
                total = len(self.train_loader)
                batch_x, batch_y = (item.cuda() for item in data)
                self.optimizer.zero_grad()
                logit_original = self.model(batch_x)
                l = loss(logit_original, batch_y)
                l.backward()
                self.optimizer.step()
                corrects = (torch.max(logit_original, 1)[1].view(batch_y.size()).data == batch_y.data).sum()
                accuracy = 100 * corrects / len(batch_y)
                print(f'Batch[{i + 1}/{total}] - loss: {l.item():.6f}  accuracy: {accuracy:.4f}%({corrects}/{batch_y.size(0)})')
            # val
            self.model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for x, y in self.val_loader:
                    x = x.cuda()
                    logits = self.model(x)
                    preds = torch.argmax(logits, dim=1)
                    all_preds.extend(preds.cpu().tolist())
                    all_labels.extend(y.tolist())

                acc = accuracy_score(all_labels, all_preds)
                print(f"Validation Accuracy: {acc:.4f}")
                if acc > self.best_acc:
                   self.best_acc = acc
                   os.makedirs(os.path.dirname(self.model_path), exist_ok=True)
                   torch.save({
                        'model_state_dict': self.model.state_dict()
                    }, self.model_path)
                   print(f"Model saved at {self.model_path}")
                print("Best val set acc:", self.best_acc)


    # test
    def test(self, loader, test_name=""):
       self.model.eval()
       all_preds, all_labels = [], []
       start_time = time.time()
       print(f"> {test_name} Start test!")

       with torch.no_grad():
         for i,data in enumerate(loader):
            t0 = time.time()
            x,y = (item.cuda(non_blocking=True) for item in data)
            t1 = time.time()
            logits = self.model(x)
            t2 = time.time()
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.tolist())
            t3 = time.time()
            print(f"[Batch {i}] | load: {t1 - t0:.3f}s | forward: {t2 - t1:.3f}s | predict: {t3 - t2:.3f}s ")
       report = classification_report(
           all_labels, all_preds,
           labels=[0, 1],
           target_names=['NR', 'FR'],
           digits=3,
           output_dict=True
       )
       print(f"{test_name} result: {report['accuracy']:.4f}")
       total_time = time.time() - start_time
       print(f"> {test_name} Finished Spend: {total_time:.2f}s in total")
       return report


#1.6 Run

In [15]:
# trainset : (80:80)*20 / trainlabel
# valset : 8:8
# origin_test : 242:242
# target_test : 1000:1000
def Runall():
  model = Classifier(train_loader, val_loader)
  model.train_val()
  res_o = model.test(test_loader_origin, "Origin")
  res_t = model.test(test_loader_target, "Target")
  if res_o['accuracy'] is not None:
      print(f"Origin: Accuracy: {res_o['accuracy']:.4f} | "
      f"F1 Score: {res_o['macro avg']['f1-score']:.4f} | "
      f"Precision: {res_o['macro avg']['precision']:.4f} | "
      f"Recall: {res_o['macro avg']['recall']:.4f}")
  if res_t['accuracy'] is not None:
      print(f"Target: Accuracy: {res_t['accuracy']:.4f} | "
      f"F1 Score: {res_t['macro avg']['f1-score']:.4f} | "
      f"Precision: {res_t['macro avg']['precision']:.4f} | "
      f"Recall: {res_t['macro avg']['recall']:.4f}")
Runall()


Found saved model at /content/drive/MyDrive/Classifier_model/best_model.pth, loading...
Skipping training, using loaded model.
> Origin Start test!
[Batch 0] | load: 0.000s | forward: 0.002s | predict: 0.083s 
[Batch 1] | load: 0.000s | forward: 0.001s | predict: 0.084s 
Origin result: 0.8512
> Origin Finished Spend: 3.72s in total
> Target Start test!
[Batch 0] | load: 0.000s | forward: 0.002s | predict: 0.039s 
[Batch 1] | load: 0.000s | forward: 0.001s | predict: 0.039s 
[Batch 2] | load: 0.000s | forward: 0.001s | predict: 0.034s 
[Batch 3] | load: 0.000s | forward: 0.002s | predict: 0.034s 
[Batch 4] | load: 0.000s | forward: 0.001s | predict: 0.036s 
[Batch 5] | load: 0.000s | forward: 0.002s | predict: 0.037s 
[Batch 6] | load: 0.000s | forward: 0.001s | predict: 0.034s 
[Batch 7] | load: 0.000s | forward: 0.001s | predict: 0.035s 
[Batch 8] | load: 0.000s | forward: 0.001s | predict: 0.034s 
[Batch 9] | load: 0.000s | forward: 0.001s | predict: 0.036s 
[Batch 10] | load: 0.000s

# 2. CNN-Based VAE

#2.1 Dataloader

In [8]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Subset
import shutil
import torch.nn.functional as F
from collections import defaultdict
def img_transform():
    return transforms.Compose([
        transforms.Lambda(lambda img: img.crop((0, 100, 768, 400)) if img.size[0] >= 768 and img.size[1] >= 400 else img),
        transforms.Resize((256,256)),
        transforms.ToTensor()
    ])

def get_limited_indices(dataset, target_labels, max_per_class):
    label_counts = defaultdict(int)
    selected_indices = []

    for idx, label in enumerate(dataset.targets):
        if label in target_labels and label_counts[label] < max_per_class:
            selected_indices.append(idx)
            label_counts[label] += 1

        # early stop
        if all(label_counts[l] >= max_per_class for l in target_labels):
            break

    return selected_indices

if os.path.exists('/content/OCT5K'):
    shutil.rmtree('/content/OCT5K')
shutil.copytree('/content/drive/MyDrive/OCT5K', '/content/OCT5K')
lp_target = '/content/OCT5K'
transform = img_transform()
origin_dataset = ImageFolder(root=loadpath, transform=transform)
target_dataset = ImageFolder(root=lp_target, transform=transform)
print(origin_dataset.class_to_idx)
print(target_dataset.class_to_idx)
# deal with iamgefolder issue: it treat each folder as an auto label for one class
# but we do binary, since dme/normal is the maximum intersection between two datasets
max_images_per_class = 1000
origin_indices = get_limited_indices(origin_dataset, target_labels=[1, 3], max_per_class=max_images_per_class)
target_indices = get_limited_indices(target_dataset, target_labels=[1, 2], max_per_class=max_images_per_class)
origin_subset = Subset(origin_dataset, origin_indices)
target_subset = Subset(target_dataset, target_indices)

{'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}
{'AMD': 0, 'DME': 1, 'Normal': 2}


In [9]:
origin_loader = DataLoader(origin_subset, batch_size=16, shuffle=True, num_workers=2)
target_loader = DataLoader(target_subset, batch_size=16, shuffle=True, num_workers=2)

#2.2 VAE Definition

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.skip = nn.Conv2d(in_channels, out_channels, kernel_size=1) \
            if in_channels != out_channels else nn.Identity()
        self.activation = nn.GELU()

    def forward(self, x):
        return self.activation(self.block(x) + self.skip(x))


class Encoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.encoder = nn.Sequential(
            ResidualBlock(3, 32),     # (B,3,256,256) -> (B,32,256,256)
            nn.MaxPool2d(2),          # -> (B,32,128,128)

            ResidualBlock(32, 64),    # -> (B,64,128,128)
            nn.MaxPool2d(2),          # -> (B,64,64,64)

            ResidualBlock(64, 128),   # -> (B,128,64,64)
            nn.MaxPool2d(2),          # -> (B,128,32,32)

            ResidualBlock(128, 256),  # -> (B,256,32,32)
            nn.MaxPool2d(2),          # -> (B,256,16,16)

            nn.Conv2d(256, latent_dim, kernel_size=1),  # -> (B,latent_dim,16,16)
            nn.AdaptiveAvgPool2d((1, 1))                # -> (B,latent_dim,1,1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)  # -> (B, latent_dim)



class Decoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 256 * 8 * 8)

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # -> (128,16,16)
            ResidualBlock(128, 128),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),   # -> (64,32,32)
            ResidualBlock(64, 64),

            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),    # -> (32,64,64)
            ResidualBlock(32, 32),

            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),    # -> (16,128,128)
            ResidualBlock(16, 16),

            nn.ConvTranspose2d(16, 3, kernel_size=4, stride=2, padding=1),     # -> (3,256,256)
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)                        # -> (B,256*8*8)
        x = x.view(x.size(0), 256, 8, 8)      # -> (B,256,8,8)
        return self.decoder(x)


class VAE(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.encoder = Encoder(latent_dim=latent_dim)
        self.fc_mu = nn.Linear(latent_dim, latent_dim)
        self.fc_logvar = nn.Linear(latent_dim, latent_dim)
        self.decoder_input = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.GELU()  # 或 GELU
        )
        self.decoder = Decoder(latent_dim=latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        z = self.encoder(x)
        mu = self.fc_mu(z)
        logvar = self.fc_logvar(z)
        z_sample = self.reparameterize(mu, logvar)
        x_recon = self.decoder(self.decoder_input(z_sample))
        return x_recon, mu, logvar




#2.3 VAE Trainer Definition

In [13]:
import torchvision.models as models
from torchmetrics.image.fid import FrechetInceptionDistance

class VGGPerceptualLoss(nn.Module):
    def __init__(self, layers=['relu3_3'], resize=True):
        super().__init__()
        vgg = models.vgg16(pretrained=True).features.eval()
        self.resize = resize

        self.selected_layers = {
            'relu1_2': 3,
            'relu2_2': 8,
            'relu3_3': 15,
            'relu4_3': 22,
        }
        self.layer_index = self.selected_layers[layers[0]]

        self.vgg_part = nn.Sequential(*list(vgg.children())[:self.layer_index + 1])
        for p in self.vgg_part.parameters():
            p.requires_grad = False

        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).cuda()
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).cuda()

    def forward(self, x, y):

        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
            y = y.repeat(1, 3, 1, 1)

        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std

        feat_x = self.vgg_part(x)
        feat_y = self.vgg_part(y)
        loss = nn.functional.l1_loss(feat_x, feat_y)
        return loss


class VAETrainer:
    def __init__(self, model, dataloader, save_path_1, save_path_2,
                 lr=2e-4, device='cuda', beta=1.0,
                 recon_type='mse'):
        self.model = model.to(device)
        self.dataloader = dataloader
        self.save_path_1 = save_path_1
        self.save_path_2 = save_path_2

        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        self.device = device
        self.beta = beta
        self.recon_type = recon_type
        self.percep_loss = VGGPerceptualLoss().to(self.device)
        self.fid_metric = FrechetInceptionDistance(feature=2048).to(self.device)

    def loss_f(self, recon_x, x, mu, logvar):
        if self.recon_type == 'mse':
            recon_loss = F.mse_loss(recon_x, x, reduction='mean')
        elif self.recon_type == 'l1':
            recon_loss = F.l1_loss(recon_x, x, reduction='mean')
        else:
            raise ValueError("recon_type must be 'mse' or 'l1'")

        kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        L1_loss = F.l1_loss(recon_x, x, reduction='mean')
        percep = self.percep_loss(recon_x, x)
        total_loss = 0.9 * recon_loss + 0.1 * L1_loss +  0.2 * percep + self.beta * kl_div
        return total_loss, recon_loss.item(), kl_div.item(), percep.item()

    def train(self, num_epochs=50, verbose=True):
        print("Start training!")
        val_time = time.time()
        self.model.train()
        for epoch in range(1, num_epochs + 1):
            self.beta = float(1 / (1 + np.exp(-0.1 * (epoch - 50))))
            total_loss = total_recon = total_kl = total_percep =0
            epoch_time = time.time()
            print(f"Epoch {epoch}/{num_epochs} Start, lastest epoch time cost:{epoch_time-val_time:.5f} seconds")
            for x, _ in self.dataloader:
                x = x.to(self.device)
                x_recon, mu, logvar = self.model(x)
                loss, recon_loss, kl_loss, percep_loss = self.loss_f(x_recon, x, mu, logvar)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
                total_recon += recon_loss
                total_kl += kl_loss
                total_percep += percep_loss
            val_time = time.time()
            if verbose:
                print(f"Epoch {epoch}/{num_epochs} | Loss: {total_loss:.4f} | Recon: {total_recon:.4f} | KL: {total_kl:.4f} | Percep: {total_percep:.4f}")
            # FID Evaluation
            with torch.no_grad():
                self.fid_metric.reset()
                for x, _ in self.dataloader:
                    x = x.to(self.device)
                    x_recon, _, _ = self.model(x)
                    real_uint8 = (x.clamp(0,1)* 255).to(torch.uint8)
                    fake_uint8 = (x_recon.clamp(0,1) * 255).to(torch.uint8)
                    self.fid_metric.update(real_uint8, real=True)
                    self.fid_metric.update(fake_uint8, real=False)
                    break
                fid_score = self.fid_metric.compute().item()
                print(f"FID score: {fid_score:.2f}")

        torch.save(self.model.encoder.state_dict(), self.save_path_1)
        torch.save(self.model.decoder.state_dict(), self.save_path_2)
        print(f"Encoder saved to {self.save_path_1}")
        print(f"Decoder saved to {self.save_path_2}")




# 2.4 Train on both domain and save param
1.1->1.3->2.1->2.2->2.3

In [ ]:
# set load path // origin_loader , target_loader
clear_memory()
save_path = '/content/drive/MyDrive/VAE_models'
VAEoencoder = os.path.join(save_path, 'VAE_O','encoder.pth')
VAEodecoder = os.path.join(save_path, 'VAE_O','decoder.pth')
VAEtencoder = os.path.join(save_path, 'VAE_T','encoder.pth')
VAEtdecoder = os.path.join(save_path, 'VAE_T','decoder.pth')

latent_dim = 32
vae_origin = VAE(latent_dim=latent_dim)
trainer_o = VAETrainer(
    model=vae_origin,
    dataloader=origin_loader,
    save_path_1= VAEoencoder,
    save_path_2= VAEodecoder,
    lr=2e-4,
    device=device,
    recon_type='mse'
)


trainer_o.train(num_epochs=50, verbose=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start training!
Epoch 1/50 Start, lastest epoch time cost:0.00053 seconds


In [ ]:
clear_memory()
vae_target = VAE(latent_dim=latent_dim)
trainer_t = VAETrainer(
    model=vae_target,
    dataloader=target_loader,
    save_path_1= VAEtencoder,
    save_path_2= VAEtdecoder,
    lr=2e-4,
    device=device,
    recon_type='mse'
)


trainer_t.train(num_epochs=6, verbose=True)

# 2.5 Target domain reconstruction by VAE

In [ ]:
import os
import torchvision.utils as vutils
from torchvision.transforms.functional import to_pil_image
# 1->DME / 2->Normal
def image_recon_eval(save_root="/content/recon_results", n_per_class=1000):
    if os.path.exists(save_root):
        shutil.rmtree(save_root)
    os.makedirs(save_root)
    vae_cross = VAE(latent_dim=latent_dim).to(device)
    vae_cross.encoder.load_state_dict(torch.load(VAEtencoder, map_location=device))
    vae_cross.decoder.load_state_dict(torch.load(VAEodecoder, map_location=device))
    vae_cross.eval()

    os.makedirs(save_root, exist_ok=True)
    class_names = target_loader.dataset.dataset.classes  # ImageFolder
    save_counter = {cls_name: 0 for cls_name in class_names}

    with torch.no_grad():
        for x, y in target_loader:
            x = x.to(device)
            x_recon, _, _ = vae_cross(x)

            for i in range(x.size(0)):
                label = y[i].item()
                cls_name = class_names[label]
                if save_counter[cls_name] >= n_per_class:
                    continue

                cls_path = os.path.join(save_root, cls_name)
                os.makedirs(cls_path, exist_ok=True)

                recon_img = to_pil_image(x_recon[i].cpu())
                recon_img.save(os.path.join(cls_path, f"{save_counter[cls_name]}_recon.png"))

                save_counter[cls_name] += 1

            if all(count >= n_per_class for count in save_counter.values()):
                print(f"Save {n_per_class} images for each class, {n_per_class * len(class_names)} in total")
                return
#Run
image_recon_eval(save_root="/content/recon", n_per_class=100)

